<a href="https://colab.research.google.com/github/LouisVanLangendonck/UPC-AML-ArchitectureClassif/blob/main/SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SVM models on extracted features

This file contains three main sections:
 1. SVM models on single kernels (linear, rbf, polynomial) and weighted sums and products 
 2. SVM models for concatenated vectors (a concatenated vector of 4 or 5 feature vectors is considered)
 3. SVM models obtained using MKLpy package (used for direct sum of kernels, i.e. kernels constructed from different feature spaces and then added together).

# 1. SVM models on single kernels, weighted sums, and products

## Imports and preparation

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

In [ ]:
PATH = "/content/drive/MyDrive/aml/models/extracted_features/inception_resnet_v2.npy"

In [ ]:
def load_extracted_feats(PATH):
  
  """
    loads extracted features via PATH
  """

  np_load_old = np.load
  # modify the default parameters of np.load
  np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

  (train_features, train_labels), (test_features, test_labels) = np.load(PATH)

  return train_features, train_labels, test_features, test_labels

  def sets_for_SVM(size_train, size_test, train_features, train_labels, test_features, test_labels):

  """
    Returns train, val,  test set with appropiate sizes

  """
  tf = train_features
  
  tl = [0 for j in range(len(train_labels))]
  for i in range(len(train_labels)):
    for idx, val in enumerate(train_labels[i, ]):
      if val == 1:
        tl[i] = idx + 1

  tl = np.array(tl)

  test_f = test_features

  test_l = [0 for j in range(len(test_labels))]
  for i in range(len(test_labels)):
    for idx, val in enumerate(test_labels[i, ]):
      if val == 1:
        test_l[i] = idx + 1


  return tf, tl, test_f, test_l

def separation_train_val(train_features, train_labels):
  """
    There is an initla separation between train and validation. This validation 
    set is so it is possible to compare all models (NN, SVMs...)
  """

  inp_train_features, inp_val_features, train_labels, val_labels = train_test_split(train_features, train_labels, stratify=train_labels, shuffle=True,
                                                    test_size=0.20, 
                                                    random_state=1)
  return inp_train_features, inp_val_features, train_labels, val_labels

In [ ]:
import numpy as np
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix  
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.utils.fixes import loguniform
from sklearn.metrics import f1_score
from time import time
import time
from matplotlib.colors import Normalize
import os
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.metrics.pairwise import polynomial_kernel

import matplotlib.pyplot as plt
import pickle 

In [ ]:
train_features, train_labels, test_features, test_labels = load_extracted_feats(PATH)
size_train = train_features.shape[0]
size_test = test_features.shape[0]
train_features, train_labels, test_features, test_labels = sets_for_SVM(size_train, size_test, train_features, train_labels, test_features, test_labels)
train_features, val_features, train_labels, val_labels = separation_train_val(train_features, train_labels)
with open('/content/drive/MyDrive/aml/models/extracted_features/class_encoding.pkl', 'rb') as f:
  class_encoding = pickle.load(f)

## SVM models (single linear, gaussian rbf, polynomial)

In [ ]:
# Linear kernel
C = [1e-4, 1e-3, 1e-2, 1e-1, 1, 10]
svm_val_score = np.zeros(len(C))
for i, l in enumerate(C):
    svm = LinearSVC(C=l, class_weight="balanced", loss="hinge")
    svm.fit(train_features, train_labels)
    val_pred = svm.predict(val_features)
    svm_val_score[i] = f1_score(val_labels, val_pred, average='weighted')
    print("C= "+str(l) + " done")

# show validation scores
for i, l in enumerate(C):
    print(f'\t C={l}; validation_score (Weighted): {svm_val_score[i]}')
print()
    
# select winner..
best = np.argmax(svm_val_score)
best_C = C[best]
print(f'Best C in SVM is {best_C} with score {svm_val_score[best]}')


In [ ]:
C = [0.01, 0.1, 1, 10,100]
gamma = [1e-4, 1e-3, 1e-2, 1e-1]
svm_val_score = np.zeros((len(C), len(gamma)))
for i, l in enumerate(C):
  for j, m in enumerate(gamma):
      svm = SVC(C=l, gamma=m, kernel="rbf", class_weight="balanced")
      svm.fit(train_features, train_labels)
      val_pred = svm.predict(val_features)
      svm_val_score[i,j] = f1_score(val_labels, val_pred, average='weighted')
      print("C= "+str(l) + "gamma = " + str(m) +" done")


# show validation scores
for i, l in enumerate(C):
  for j, m in enumerate(gamma):
    print(f'\t C={l} gamma={m}; validation_score: {svm_val_score[i,j]}')
print()
    


In [ ]:
C = [0.01, 0.1, 1, 10]
coef0 = [0, 1, 10]

d = [2,3,4]
svm_val_score_weighted = np.zeros((len(C), len(coef0), len(d)))
for i, l in enumerate(C):
  for j, m in enumerate(coef0):
    for k, n in enumerate(d):
      svm = SVC(C=l, coef0=m, degree=n, kernel="poly", class_weight="balanced")
      svm.fit(train_features, train_labels)
      val_pred = svm.predict(val_features)
      svm_val_score_weighted[i,j,k] = f1_score(val_labels, val_pred, average='weighted')
      print("C= "+str(l) + "c = " + str(m) + "degree" + str(n) + "done")


# show validation scores
for i, l in enumerate(C):
  for j, m in enumerate(coef0):
    for k,n in enumerate(d):
      print(f'\t C={l} c={m} degree={n}; weighted_score: {svm_val_score_weighted[i,j,k]}')
print()

## SVM models (weighted sum and product of kernels)

In [ ]:
# Linear + RBF

max_val_score = 0
max_wei_score = 0
param = [0,0,0]

klinear =  np.dot(train_features, train_features.T)
klinear_val = np.dot(val_features, train_features.T)

gamma = [0.001, 0.005, 0.01, 0.05, 0.1]

for gi in gamma:
  krbf = rbf_kernel(train_features, train_features, gamma=gi)
  krbf_val = rbf_kernel(val_features, train_features, gamma=gi)
  
  print("------------------------------------------------------------------------------")
  print("NOW doing gamma= " + str(gi))
  b = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
  for bi in b:
    print("----------------------------------------------")
    print("NOW doing b= " + str(bi))
    k = bi*klinear + (1-bi)*krbf
    kv = bi*klinear_val + (1-bi)*krbf_val

    C = [1e-2, 1e-1, 1, 10]
    svm_val_score = np.zeros(len(C))
    for i, l in enumerate(C):
        svm = SVC(kernel="precomputed", C=l, class_weight="balanced")
        svm.fit(k, train_labels)
        val_pred = svm.predict(kv)

        if f1_score(val_pred, val_labels, average='weighted') >= max_wei_score:
            max_val_score = f1_score(val_labels, val_pred, average='micro')
            max_wei_score = f1_score(val_labels, val_pred, average='weighted')
            
            param[0] = gi
            param[1] = bi
            param[2] = l

        print("C= "+str(l) + " done")

print(max_val_score)
print(max_wei_score)
print(param)


In [ ]:
# Polynomial + RBF
max_val_score = 0
max_wei_score = 0
param = [0,0,0,0,0]

gamma = [0.001, 0.005, 0.01, 0.05,0.1]
d  = [2,3,4]
coef0 = [0,1]

for gi in gamma:
  krbf = rbf_kernel(train_features, train_features, gamma = gi)
  krbf_val = rbf_kernel(val_features, train_features, gamma= gi)

  for di in d:
    for ci in coef0:
      print("----------------------------------------------------------------------------------------------")
      print("PARAMETERS gamma = " + str(gi) + " d = " + str(di) + " coef0 = " + str(ci))
      kpoly =  polynomial_kernel(train_features, train_features, degree=di, coef0 = ci)
      kpoly_val = polynomial_kernel(val_features, train_features, degree=di, coef0 =ci)
      b = [0.1, 0.3, 0.5, 0.7, 0.9]
      for bi in b:
        print("----------------------------------------------")
        print("NOW doing b= " + str(bi))
        k = bi*kpoly + (1-bi)*krbf
        kv = bi*kpoly_val + (1-bi)*krbf_val

        C = [1e-2, 1e-1, 1, 10]
        svm_val_score = np.zeros(len(C))
        for i, l in enumerate(C):
            svm = SVC(kernel="precomputed", C=l,class_weight="balanced")
            svm.fit(k, train_labels)
            val_pred = svm.predict(kv)
            

            if f1_score(val_pred, val_labels, average='weighted') >= max_wei_score:
              max_wei_score = f1_score(val_labels, val_pred, average='weighted')
              max_val_score = f1_score(val_labels, val_pred, average='micro')

              param[0] = gi
              param[1] = di
              param[2] = ci
              param[3] = bi
              param[4] = l

        print("---------------------------------------------")

print(max_val_score)
print(max_wei_score)
print(param)

In [ ]:
# Linear * RBF

max_val_score = 0
max_wei_score = 0
param = [0,0]

klinear =  np.dot(train_features, train_features.T)
klinear_val = np.dot(val_features, train_features.T)

gamma = [0.001, 0.005, 0.01, 0.05, 0.1]

for gi in gamma:
  krbf = rbf_kernel(train_features, train_features, gamma=gi)
  krbf_val = rbf_kernel(val_features, train_features, gamma=gi)
  
  print("------------------------------------------------------------------------------")
  print("NOW doing gamma= " + str(gi))
  k = np.multiply(klinear, krbf)
  kv = np.multiply(klinear_val, krbf_val)
  C = [1e-2, 1e-1, 1, 10]
  svm_val_score = np.zeros(len(C))
  for i, l in enumerate(C):
      svm = SVC(kernel="precomputed", C=l,class_weight="balanced")
      svm.fit(k, train_labels)
      val_pred = svm.predict(kv)

      if f1_score(val_pred, val_labels, average='weighted') >= max_wei_score:
          max_val_score = f1_score(val_labels, val_pred, average='micro')
          max_wei_score = f1_score(val_labels, val_pred, average='weighted')
          
          param[0] = gi
          param[1] = l

      print("C= "+str(l) + " done")

print(max_val_score)
print(max_wei_score)
print(param)

In [ ]:
#RBF * Polynomial

max_val_score = 0
max_wei_score = 0
param = [0,0,0,0]

gamma = [0.001, 0.005, 0.01, 0.05,0.1]
d  = [2,3,4]
coef0 = [0,1]

for gi in gamma:
  krbf = rbf_kernel(train_features, train_features, gamma = gi)
  krbf_val = rbf_kernel(val_features, train_features, gamma= gi)

  for di in d:
    for ci in coef0:
      print("----------------------------------------------------------------------------------------------")
      print("PARAMETERS gamma = " + str(gi) + " d = " + str(di) + " coef0 = " + str(ci))
      kpoly =  polynomial_kernel(train_features, train_features, degree=di, coef0 = ci)
      kpoly_val = polynomial_kernel(val_features, train_features, degree=di, coef0 =ci)
      
      k = np.multiply(krbf, kpoly)
      kv = np.multiply(krbf_val, kpoly_val)

      C = [1e-2, 1e-1, 1, 10]
      svm_val_score = np.zeros(len(C))
      for i, l in enumerate(C):

          svm = SVC(kernel="precomputed", C=l, class_weight="balanced")
          svm.fit(k, train_labels)
          val_pred = svm.predict(kv)
         
          print("C= "+str(l) + " done")

          if f1_score(val_pred, val_labels, average='weighted') >= max_wei_score:
            max_wei_score = f1_score(val_labels, val_pred, average='weighted')
            max_val_score = f1_score(val_labels, val_pred, average='micro')

            param[0] = gi
            param[1] = di
            param[2] = ci
            param[3] = l

      print("---------------------------------------------")
print(max_val_score)
print(max_wei_score)
print(param)

In [ ]:
# RBF * RBF
max_val_score = 0
max_wei_score = 0
param = [0,0]



gamma = [0.001, 0.005, 0.01, 0.05 ,0.1]
for gi in gamma:
  krbf1 = rbf_kernel(train_features, train_features, gamma=gi)
  krbf1_val = rbf_kernel(val_features, train_features, gamma=gi)
  krbf2 = rbf_kernel(train_features, train_features, gamma=gi)
  krbf2_val = rbf_kernel(val_features, train_features, gamma=gi)
  
  print("------------------------------------------------------------------------------")
  print("NOW doing gamma= " + str(gi))
  k = np.multiply(krbf1, krbf2)
  kv = np.multiply(krbf1_val, krbf2_val)
  C = [1e-2, 1e-1, 1, 10]
  svm_val_score = np.zeros(len(C))
  for i, l in enumerate(C):
      svm = SVC(kernel="precomputed", C=l,class_weight="balanced")
      svm.fit(k, train_labels)
      val_pred = svm.predict(kv)

      if f1_score(val_pred, val_labels, average='weighted') >= max_wei_score:
          max_val_score = f1_score(val_labels, val_pred, average='micro')
          max_wei_score = f1_score(val_labels, val_pred, average='weighted')
          
          param[0] = gi
          param[1] = l
          
      print("C= "+str(l) + " done")

print(max_val_score)
print(max_wei_score)
print(param)

# 2. SVM models for concatenated vectors

## Imports and preparation

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

def load_extracted_feats(PATH):
  
  """
    loads extracted features via PATH
  """

  np_load_old = np.load
  # modify the default parameters of np.load
  np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

  (train_features, train_labels), (test_features, test_labels) = np.load(PATH)

  return train_features, train_labels, test_features, test_labels
def sets_for_SVM(size_train, size_test, train_features, train_labels, test_features, test_labels):

  """
    Returns train, val,  test set with appropiate sizes

  """

  tf = train_features
  
  #tf = np.resize(train_features, (size_train, 2048)) 
  tl = [0 for j in range(len(train_labels))]
  for i in range(len(train_labels)):
    for idx, val in enumerate(train_labels[i, ]):
      if val == 1:
        tl[i] = idx + 1

  tl = np.array(tl)


  #test_f = np.resize(test_features, (size_test, 7*7*512))
  test_f = test_features

  test_l = [0 for j in range(len(test_labels))]
  for i in range(len(test_labels)):
    for idx, val in enumerate(test_labels[i, ]):
      if val == 1:
        test_l[i] = idx + 1


  return tf, tl, test_f, test_l


def separation_train_val(train_features, train_labels):
"""
  There is an initla separation between train and validation. This validation 
  set is so it is possible to compare all models (NN, SVMs...)
"""

  inp_train_features, inp_val_features, train_labels, val_labels = train_test_split(train_features, train_labels, stratify=train_labels, shuffle=True,
                                                    test_size=0.20, 
                                                    random_state=1)
  return inp_train_features, inp_val_features, train_labels, val_labels


In [ ]:
import numpy as np
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix  
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.utils.fixes import loguniform
from sklearn.metrics import f1_score
from time import time
import os
import time
from matplotlib.colors import Normalize
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.metrics.pairwise import polynomial_kernel
import seaborn as sns


import matplotlib.pyplot as plt
import pickle 

In [ ]:
concat_path_base = '/content/drive/MyDrive/aml/models/extracted_features/'
concat_path_list = ['inception_resnet_v2.npy', 'vgg19.npy', 'xception.npy', 'SIFT.npy', 'efficientnetb7.npy']
train_features = []
test_features = []

np_load_old = np.load
  # modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

for idx, feature_file in enumerate(concat_path_list):
    feature_path = os.path.join(concat_path_base, feature_file)
    (train_features_to_concat, train_labels), (test_features_to_concat, test_labels) = np.load(feature_path)
    if idx == 0:
      train_features = train_features_to_concat
      test_features = test_features_to_concat
    else:
      train_features=np.append(train_features, train_features_to_concat, 1)
      test_features=np.append(test_features, test_features_to_concat, 1)

In [ ]:
size_train = train_features.shape[0]
size_test = test_features.shape[0]
train_features, train_labels, test_features, test_labels = sets_for_SVM(size_train, size_test, train_features, train_labels, test_features, test_labels)
train_features, val_features, train_labels, val_labels = separation_train_val(train_features, train_labels)
with open('/content/drive/MyDrive/aml/models/extracted_features/class_encoding.pkl', 'rb') as f:
  class_encoding = pickle.load(f)

## Linear, Gaussian RBF,Polynomial (for concatenated vector)

In [ ]:
# Linear Kernel

C = [1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
svm_val_score = np.zeros(len(C))
for i, l in enumerate(C):
    svm = LinearSVC(C=l, class_weight="balanced")
    svm.fit(train_features, train_labels)
    val_pred = svm.predict(val_features)
    svm_val_score[i] = f1_score(val_labels, val_pred, average='weighted')
    print("C= "+str(l) + " done")

# show validation scores
for i, l in enumerate(C):
    print(f'\t C={l}; validation_score (weighted): {svm_val_score[i]}')
print()
    
# select winner..
best = np.argmax(svm_val_score)
best_C = C[best]
print(f'Best C in SVM is {best_C} with score {svm_val_score[best]}')

In [ ]:
# RBF Kernel
C = [1e-4, 1e-3, 1e-2, 1e-1, 1, 10]
gamma = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1]
svm_val_score = np.zeros((len(C), len(gamma)))
for i, l in enumerate(C):
  for j, m in enumerate(gamma):
      svm = SVC(C=l, gamma=m, kernel="rbf", class_weight="balanced")
      svm.fit(train_features, train_labels)
      val_pred = svm.predict(val_features)
      svm_val_score[i,j] = f1_score(val_labels, val_pred, average='weighted')
      print("C= "+str(l) + "gamma = " + str(m) +" done")


# show validation scores
for i, l in enumerate(C):
  for j, m in enumerate(gamma):
    print(f'\t C={l} gamma={m}; validation_score: {svm_val_score[i,j]}')
print()

In [ ]:
# Polynomial

C = [1, 10, 100]
coef0 = [0, 1]
d = [2,3,4]
svm_val_score = np.zeros((len(C), len(coef0), len(d)))
for i, l in enumerate(C):
  for j, m in enumerate(coef0):
    for k, n in enumerate(d):
      svm = SVC(C=l, coef0=m, degree=n, kernel="poly")
      svm.fit(train_features, train_labels)
      val_pred = svm.predict(val_features)
      svm_val_score[i,j,k] = f1_score(val_labels, val_pred, average='weighted')
      print("C= "+str(l) + "c = " + str(m) + "degree" + str(n) + "done")


# show validation scores
for i, l in enumerate(C):
  for j, m in enumerate(coef0):
    for k,n in enumerate(d):
      print(f'\t C={l} c={m} degree={n}; validation_score: {svm_val_score[i,j,k]}')
print()

# 3.SVM with a kernel obtained via MKL

## Imports and preparation

In [ ]:
!pip install MKLpy
import MKLpy


In [ ]:
def sets_for_SVM(size_train, size_test, train_features, train_labels, test_features, test_labels):

  """
    Returns train, val,  test set with appropiate sizes

  """
  tf = train_features
  
  #tf = np.resize(train_features, (size_train, 2048)) 
  tl = [0 for j in range(len(train_labels))]
  for i in range(len(train_labels)):
    for idx, val in enumerate(train_labels[i, ]):
      if val == 1:
        tl[i] = idx + 1

  tl = np.array(tl)


  #test_f = np.resize(test_features, (size_test, 7*7*512))
  test_f = test_features

  test_l = [0 for j in range(len(test_labels))]
  for i in range(len(test_labels)):
    for idx, val in enumerate(test_labels[i, ]):
      if val == 1:
        test_l[i] = idx + 1


  return tf, tl, test_f, test_l

def separation_train_val(train_features, train_labels):
  """
    There is an initla separation between train and validation. This validation 
    set is so it is possible to compare all models (NN, SVMs...)
  """

  inp_train_features, inp_val_features, train_labels, val_labels = train_test_split(train_features, train_labels, stratify=train_labels, shuffle=True,
                                                    test_size=0.20, 
                                                    random_state=1)
  return inp_train_features, inp_val_features, train_labels, val_labels

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')



import numpy as np
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix  
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.utils.fixes import loguniform
from sklearn.metrics import f1_score
from time import time
import os
import time
from matplotlib.colors import Normalize


import matplotlib.pyplot as plt
import pickle 

In [ ]:
summ_path = '/content/drive/MyDrive/aml/models/extracted_features/'
summ_duo = ['SIFT.npy', 'vgg19.npy', 'xception.npy', 'inception_resnet_v2.npy']


np_load_old = np.load
  # modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

feature_path = os.path.join(summ_path, summ_duo[0])
(train_features_1, train_labels_1), (test_features_1, test_labels_1) = np.load(feature_path)
feature_path = os.path.join(summ_path, summ_duo[1])
(train_features_2, train_labels_2), (test_features_2, test_labels_2) = np.load(feature_path)
feature_path = os.path.join(summ_path, summ_duo[2])
(train_features_3, train_labels_3), (test_features_3, test_labels_3) = np.load(feature_path)
feature_path = os.path.join(summ_path, summ_duo[3])
(train_features_4, train_labels_4), (test_features_4, test_labels_4) = np.load(feature_path)


size_train_1 = train_features_1.shape[0]
size_test_1 = test_features_1.shape[0]
train_features_1, train_labels_1, test_features_1, test_labels_1 = sets_for_SVM(size_train_1, size_test_1, train_features_1, train_labels_1, test_features_1, test_labels_1)

size_train_2 = train_features_2.shape[0]
size_test_2 = test_features_2.shape[0]
train_features_2, train_labels_2, test_features_2, test_labels_2 = sets_for_SVM(size_train_2, size_test_2, train_features_2, train_labels_2, test_features_2, test_labels_2)

size_train_3 = train_features_3.shape[0]
size_test_3 = test_features_3.shape[0]
train_features_3, train_labels_3, test_features_3, test_labels_3 = sets_for_SVM(size_train_3, size_test_3, train_features_3, train_labels_3, test_features_3, test_labels_3)

size_train_4 = train_features_4.shape[0]
size_test_4 = test_features_4.shape[0]
train_features_4, train_labels_4, test_features_4, test_labels_4 = sets_for_SVM(size_train_4, size_test_4, train_features_4, train_labels_4, test_features_4, test_labels_4)

with open('/content/drive/MyDrive/aml/models/extracted_features/class_encoding.pkl', 'rb') as f:
  class_encoding = pickle.load(f)


## Constructing the direct sum kernel and testing it on different algorithms

In [ ]:
from MKLpy.preprocessing import kernel_normalization
from MKLpy.model_selection import train_test_split

K1 = rbf_kernel(train_features_1, train_features_1, gamma = 0.01)
K2 = rbf_kernel(train_features_2, train_features_2, gamma = 0.01)
K3 = rbf_kernel(train_features_2, train_features_2, gamma = 0.01)
K4 = rbf_kernel(train_features_2, train_features_2, gamma = 0.01)

KL = [K1,K2, K3, K4]


In [ ]:
KLtr,KLte,Ytr,Yte = train_test_split(KL, train_labels_1, test_size=.2, random_state=1)

In [ ]:
# AverageMKL

from MKLpy.algorithms import AverageMKL
from sklearn.metrics import f1_score
C = [1e-2, 1e-1, 1, 10]
mkl_svm_val_score = np.zeros(len(C))
for i, l in enumerate(C):
    mkl = AverageMKL(SVC(C=l))
    mkl = mkl.fit(KLtr, Ytr)
    y_pred = mkl.predict(KLte)
    mkl_svm_val_score[i] = f1_score(Yte, y_pred, average='weighted')
    print("C = " +str(l) + "weighted score" + str(f1_score(Yte, y_pred, average='weighted')) )
    print("C= "+str(l) + " done")

In [ ]:
from MKLpy.algorithms import EasyMKL, KOMD	

# EASY MKL

from MKLpy.model_selection import cross_val_score
from sklearn.svm import SVC
import numpy as np
from itertools import product
svm = SVC(C=10)
clf = EasyMKL(lam=0, multiclass_strategy='ova', learner=svm).fit(KLtr,Ytr)
from MKLpy.multiclass import OneVsRestMKLClassifier, OneVsOneMKLClassifier
print ('done')
print ('the combination weights are:')
for sol in clf.solution:
	print ('(%d vs all): ' % sol, clf.solution[sol].weights)
	
#evaluate the solution
from sklearn.metrics import accuracy_score, roc_auc_score
import numpy as np
y_pred = clf.predict(KLte)					#predictions
y_score = clf.decision_function(KLte)		#rank
accuracy = accuracy_score(Yte, y_pred)
print ('Accuracy score: %.3f' % (accuracy))


In [ ]:
easy_micro = f1_score(Yte, y_pred, average='micro')
easy_weighted = f1_score(Yte, y_pred, average='weighted')
print(easy_micro)
print(easy_weighted)